In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')

In [2]:
url = "http://localhost:8056"

In [3]:
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

In [4]:
import random

In [5]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to http://localhost:8056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '62411@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 17:54:22 GMT, Fri, 18 Apr 2025 17:54:22 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '110', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"62411@example.com","name":"test","image_url":null,"id":51,"created_at":"2025-04-18T17:54:22.606434"}...


In [6]:

user_id = new_user['id']

In [7]:
user_id

51

In [8]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to http://localhost:8056/admin/users/51/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 17:54:24 GMT, Fri, 18 Apr 2025 17:54:25 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '115', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":51,"id":51,"token":"dDPYNwzNMVHrnFPR8T3lHdb2ja9UDjmAgfDBSHT4","created_at":"2025-04-18T17:54:25.246609"}...


In [9]:
user_api_key = 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'

In [10]:
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)

In [11]:
meeting_url="https://meet.google.com/qcm-yvzc-izx"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [16]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="MyBot",
    language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:8056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'qcm-yvzc-izx', 'bot_name': 'MyBot', 'language': 'es', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 17:54:45 GMT, Fri, 18 Apr 2025 17:54:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '393', 'content-type': 'application/json'}
DEBUG: Response content: {"id":62,"user_id":15,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"f3c9727b43c28ffe7340249d2ddcd23f973d1fada1a6e27de899d50083c73910","start_time":"2025-04-18T17:54:45.692886","end_time":null,"created_at":"2025-04-18T17:54:45.472542","updated_at":"2025-04-18T17:54:45.483346"}...


In [13]:
client.reconfigure_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='pt',
    task = 'translate'
    
    
)

AttributeError: 'VexaClient' object has no attribute 'reconfigure_bot'

In [15]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:8056/bots/google_meet/qcm-yvzc-izx
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 17:54:40 GMT, Fri, 18 Apr 2025 17:54:40 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [13]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:8056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 08:39:57 GMT, Fri, 18 Apr 2025 08:39:57 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '408', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":15,"user_id":15,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"75548589999aa1ee5595ad2acad9073694023957a55ead68d3613b61ce7b4987","start_time":"2025-04-18T08:39:57.167258","end_time":null,"created_at":"2025-04-18T08:39:56.887518","updated_at":"2025-04-18T08:39:56.903290"}]}...


In [14]:
meeting_id

15

In [28]:
import pandas as pd

In [29]:
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [30]:

transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')


DEBUG: Making GET request to http://localhost:8056/transcripts/google_meet/qcm-yvzc-izx
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 14:21:55 GMT, Fri, 18 Apr 2025 14:21:55 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '675', 'content-type': 'application/json'}
DEBUG: Response content: {"id":51,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","start_time":"2025-04-18T14:21:09.974110","end_time":null,"segments":[{"start":0.0,"end":8.44,"text":" Voy a hablar en ruso y veremos cómo funciona.","language":null,"created_at":null,"speaker":null,"absolute_start_time":"2025-04-18T14:21:32.252629+00:00","absolute_end_time":"2025-04-18T14:21:40.692629+00:00"},{"start":8.44,"end":10.88,"text":" ¡...


In [31]:
df.sort_values('absolute_start_time')

,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
0,0.00,8.44,Voy a hablar en ruso y veremos cómo funciona.,None,None,None,2025-04-18T14:21:32.252629+00:00,2025-04-18T14:21:40.692629+00:00
1,8.44,10.88,¡Gracias!,None,None,None,2025-04-18T14:21:40.692629+00:00,2025-04-18T14:21:43.132629+00:00


In [20]:
df

""
